<a href="https://colab.research.google.com/github/meetgandhi123/PyTorch-Basic-Concepts/blob/main/05_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

# device config.
Device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Device usage: {Device}')

# Hyper Parameter
input_size = 784 #28*28
hidden_size = 100
batch_size = 100
num_epoch = 2
num_classes = 10
learning_rate = 0.001

# Dataset
train_dataset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transforms.ToTensor())
test_dataset = torchvision.datasets.MNIST(root='./data', train=False, transform=transforms.ToTensor())

# Dataloader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

sample = iter(train_loader)
input, output = sample.next()
# print(input.shape, output.shape)
# torch.Size([100, 1, 28, 28]) torch.Size([100])

# Model
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.l1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.l2 = nn.Linear(hidden_size, num_classes)
    def forward(self, x):
        out = self.l1(x)
        out = self.relu(out)
        out = self.l2(out)
        return out
    
model = NeuralNet(input_size, hidden_size, num_classes)

# Loss and Optimizers
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

# Training loop
num_step = len(train_loader)
for epoch in range(num_epoch):
    for i, (image,labels) in enumerate(train_loader):
        # 100,1,28,28 -> 100,28*28
        image = image.reshape(-1,28*28).to(Device)
        labels = labels.to(Device)

        # Forward
        output = model(image)
        loss = criterion(output,labels)

        # Backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1)%100==0:
            print(f'Epoch: {epoch+1} / {num_epoch}, step: {i+1} / {num_step}, loss: {loss.item():.4f}')

# Testing
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    for images, labels in test_loader:
        images = images.reshape(-1,28*28).to(Device)
        labels = labels.to(Device)
        output = model(images)

        # Value, Index
        _, predictions = torch.max(output,1)
        n_samples += labels.shape[0]
        n_correct += (predictions==labels).sum().item()

    acc = 100.0 * n_correct/n_samples
    print(f'accuract: {acc}')


Device usage: cpu
Epoch: 1 / 2, step: 100 / 600, loss: 0.3580
Epoch: 1 / 2, step: 200 / 600, loss: 0.4308
Epoch: 1 / 2, step: 300 / 600, loss: 0.3156
Epoch: 1 / 2, step: 400 / 600, loss: 0.2575
Epoch: 1 / 2, step: 500 / 600, loss: 0.2301
Epoch: 1 / 2, step: 600 / 600, loss: 0.1819
Epoch: 2 / 2, step: 100 / 600, loss: 0.3010
Epoch: 2 / 2, step: 200 / 600, loss: 0.2975
Epoch: 2 / 2, step: 300 / 600, loss: 0.2897
Epoch: 2 / 2, step: 400 / 600, loss: 0.1492
Epoch: 2 / 2, step: 500 / 600, loss: 0.1777
Epoch: 2 / 2, step: 600 / 600, loss: 0.2044
accuract: 95.25
